## Import libraries

In [75]:
import re
import string
import tensorflow as tf
from transformers import BertTokenizer, TFAutoModel
from sklearn.metrics.pairwise import cosine_similarity
from datasets import load_dataset

## Load base model and tokenizer

In [76]:
tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-base-p2")
model = TFAutoModel.from_pretrained("indobenchmark/indobert-base-p2")

Some layers from the model checkpoint at indobenchmark/indobert-base-p2 were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at indobenchmark/indobert-base-p2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


## Tokenizer test

In [77]:
text_test = ['Pupuk NPK','Pupuk Nitrogen']
text_preprocessed = tokenizer(text_test, max_length=128, padding=True, truncation=True, return_tensors="tf")
text_preprocessed.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [78]:
text_preprocessed['input_ids']

<tf.Tensor: shape=(2, 5), dtype=int32, numpy=
array([[    2,  5301, 11429, 30361,     3],
       [    2,  5301, 14344,     3,     0]])>

In [79]:
text_preprocessed['token_type_ids']

<tf.Tensor: shape=(2, 5), dtype=int32, numpy=
array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])>

In [80]:
text_preprocessed['attention_mask']

<tf.Tensor: shape=(2, 5), dtype=int32, numpy=
array([[1, 1, 1, 1, 1],
       [1, 1, 1, 1, 0]])>

## Base model test

In [81]:
test_results = model(text_preprocessed)
test_results.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [82]:
test_encoded = test_results['pooler_output']
cosine_similarity([test_encoded[0]], [test_encoded[1]])

array([[0.94776154]], dtype=float32)

## Create Dataset

In [88]:
from transformers import glue_convert_examples_to_features
import tensorflow_datasets as tfds
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
data = tfds.load('glue/mrpc')
train_dataset = glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, task='mrpc')
# train_dataset = train_dataset.shuffle(100).batch(32).repeat(2)

In [90]:
data['train']

<PrefetchDataset element_spec={'idx': TensorSpec(shape=(), dtype=tf.int32, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'sentence1': TensorSpec(shape=(), dtype=tf.string, name=None), 'sentence2': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [64]:
dataset = load_dataset("LazarusNLP/stsb_mt_id", name="en")
dataset

Found cached dataset parquet (C:/Users/Teguh/.cache/huggingface/datasets/LazarusNLP___parquet/LazarusNLP--stsb_mt_id-53495c8bc04ac9ed/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 333.58it/s]


DatasetDict({
    validation: Dataset({
        features: ['domain', 'data', 'type', 'score', 'correlation', 'text_1', 'text_2'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['domain', 'data', 'type', 'score', 'correlation', 'text_1', 'text_2'],
        num_rows: 1379
    })
})

In [70]:
train_dataset = dataset['validation']
val_dataset = train_dataset = dataset['test']

In [71]:
train_sim = [i['correlation'] for i in train_dataset]
val_sim = [i['correlation'] for i in val_dataset]
train_norm_sim = [float(i)/5.0 for i in train_sim]
val_norm_sim = [float(i)/5.0 for i in val_sim]

## Build the model

In [72]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = tokenizer(text_input)
outputs = model(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).